## Supervised Learning on Simulations on PDZ Domain

In [1]:
import sys # note temporary... 
sys.path.append("..") # note temporary...

import pandas as pd
from key_interactions_finder import pycontact_processing
from key_interactions_finder import data_preperation
from key_interactions_finder import model_building
from key_interactions_finder import post_proccessing
from key_interactions_finder import pymol_projections

### PDZ Domain Peptide Bound vs Unbound impact on domain. 

In [2]:
# all input files 
in_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Datasets\PDZ_Combined"
out_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results"

pycontact_files_horizontal = ["PyContact_Per_Frame_Interactions_Block1.csv", "PyContact_Per_Frame_Interactions_Block2.csv",
                              "PyContact_Per_Frame_Interactions_Block3.csv", "PyContact_Per_Frame_Interactions_Block4.csv",
                              "PyContact_Per_Frame_Interactions_Block5.csv", "PyContact_Per_Frame_Interactions_Block6.csv",
                              "PyContact_Per_Frame_Interactions_Block7.csv", "PyContact_Per_Frame_Interactions_Block8.csv",
                              "PyContact_Per_Frame_Interactions_Block9.csv", "PyContact_Per_Frame_Interactions_Block10.csv",
                              "PyContact_Per_Frame_Interactions_Block11.csv", "PyContact_Per_Frame_Interactions_Block12.csv",
                              ]


In [3]:
# dataset prep parts. 
pycontact_dataset = pycontact_processing.PyContactInitializer(
    pycontact_files=pycontact_files_horizontal,
    multiple_files=True,
    merge_files_method="horizontal",  
    remove_false_interactions=True,
    in_dir=in_dir,
)

Your PyContact file(s) have been succefully processed.
You have 914 features and 20000 observations.
The fully processed dataframe is accesible from the '.prepared_df' class attribute.


In [4]:
pycontact_dataset.prepared_df

,1Ace 6Glu Hbond bb-sc,2Phe 6Glu Hbond bb-bb,2Phe 5Glu Hbond bb-bb,3Leu 6Glu Hbond bb-bb,3Leu 8Ile Hydrophobic sc-sc,3Leu 7Asp Hbond bb-bb,3Leu 9Pro Hydrophobic sc-sc,3Leu 55Arg Hbond bb-sc,3Leu 112Arg Hbond bb-sc,4Gly 112Arg Hbond bb-sc,...,101Phe 106Arg Other bb-sc,108Asn 115Thr Other sc-bb,101Phe 116Asn Other bb-sc,109Ser 114Val Other sc-sc,101Phe 105Ser Other bb-sc,113Ile 116Asn Hbond bb-sc,114Val 117Nme Hbond bb-bb,113Ile 117Nme Hbond bb-bb,112Arg 117Nme Other sc-sc,112Arg 116Asn Other sc-sc
0,1.43052,2.63266,1.02764,4.04431,1.46040,0.66545,0.17311,2.71112,1.83761,2.62597,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.32276,0.00000,2.24340,0.43224,1.24919,1.18507,0.14182,1.96343,4.54474,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.52178,0.00000,0.90360,2.21138,1.38761,0.50366,1.53181,2.67639,1.34092,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.17202,0.00000,4.46798,0.00000,0.00000,0.00000,0.04288,0.00000,5.04876,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,2.96892,0.00000,6.48986,0.36254,0.16546,0.00000,0.95956,1.81796,0.25805,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Create and save a target file (to load back in next step)
# 10,000 frames were used for both states  
classification_col = ["States"] + ["Bound"] * 10000 + ["Unbound"] * 10000

classifications_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Datasets\PDZ_Classifications_Combined.dat"

with open(classifications_file, "w") as file_out:
    for item in classification_col:
        file_out.write(item + "\n")

In [6]:
# Now generate an instance of the SupervisedFeatureData class and load datasets in.
supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=pycontact_dataset.prepared_df,
    target_file=classifications_file,
    is_classification=True,
    header_present=True 
)

supervised_dataset.df_processed

Your PyContact features and target variable have been succesufully merged.
You can access this dataset through the class attribute: '.df_processed'.


,Target,1Ace 6Glu Hbond bb-sc,2Phe 6Glu Hbond bb-bb,2Phe 5Glu Hbond bb-bb,3Leu 6Glu Hbond bb-bb,3Leu 8Ile Hydrophobic sc-sc,3Leu 7Asp Hbond bb-bb,3Leu 9Pro Hydrophobic sc-sc,3Leu 55Arg Hbond bb-sc,3Leu 112Arg Hbond bb-sc,...,101Phe 106Arg Other bb-sc,108Asn 115Thr Other sc-bb,101Phe 116Asn Other bb-sc,109Ser 114Val Other sc-sc,101Phe 105Ser Other bb-sc,113Ile 116Asn Hbond bb-sc,114Val 117Nme Hbond bb-bb,113Ile 117Nme Hbond bb-bb,112Arg 117Nme Other sc-sc,112Arg 116Asn Other sc-sc
0,Bound,1.43052,2.63266,1.02764,4.04431,1.46040,0.66545,0.17311,2.71112,1.83761,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bound,0.00000,0.32276,0.00000,2.24340,0.43224,1.24919,1.18507,0.14182,1.96343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bound,0.00000,0.52178,0.00000,0.90360,2.21138,1.38761,0.50366,1.53181,2.67639,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bound,0.00000,0.17202,0.00000,4.46798,0.00000,0.00000,0.00000,0.04288,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bound,0.00000,2.96892,0.00000,6.48986,0.36254,0.16546,0.00000,0.95956,1.81796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Unbound,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,Unbound,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,Unbound,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,Unbound,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
supervised_dataset.df_processed.Target.value_counts()

Bound      10000
Unbound    10000
Name: Target, dtype: int64

In [8]:
# Filtering 
supervised_dataset.reset_filtering()
supervised_dataset.filter_by_occupancy_by_class(min_occupancy=25)
supervised_dataset.filter_by_main_or_side_chain(
    main_side_chain_types_included=["sc-sc", "bb-sc", "sc-bb", "bb-bb"] 
)
supervised_dataset.filter_by_avg_strength(average_strength_cut_off=0.5)
print(f"Number of features after filtering by average interaction scores: {len(supervised_dataset.df_filtered.columns)}")

Number of features after filtering by average interaction scores: 267


In [9]:
supervised_dataset.df_filtered

,Target,9Pro 93Tyr Hbond bb-sc,99Ser 113Ile Other sc-sc,99Ser 107Val Hbond sc-sc,99Ser 106Arg Hbond sc-bb,99Ser 105Ser Hbond sc-sc,99Ser 102Glu Hbond bb-bb,98Tyr 113Ile Hydrophobic sc-sc,98Tyr 102Glu Hbond sc-sc,98Tyr 101Phe Hbond bb-bb,...,106Arg 115Thr Hbond sc-bb,106Arg 114Val Hbond bb-bb,106Arg 113Ile Other bb-bb,105Ser 116Asn Hbond bb-sc,105Ser 115Thr Hbond sc-bb,105Ser 114Val Hbond bb-bb,105Ser 113Ile Other sc-sc,104Asn 116Asn Hbond bb-sc,104Asn 115Thr Hbond bb-bb,102Glu 105Ser Hbond sc-sc
0,Bound,2.39497,0.05120,1.03383,0.45765,1.18131,2.57110,0.76023,11.76906,5.00314,...,0.00000,9.85522,1.79038,1.19680,0.02331,2.46106,0.18907,8.34394,4.09082,1.06104
1,Bound,2.18564,1.34528,1.41214,4.20515,6.34059,1.03399,1.26384,12.16969,3.16704,...,0.93342,12.89773,2.79299,0.33323,0.23059,2.45911,0.30062,11.78538,4.27449,2.87460
2,Bound,2.87895,0.26191,1.00365,0.59664,7.36063,1.69915,2.67010,13.64869,2.65958,...,1.64280,10.75298,2.21471,2.70596,0.01028,1.79372,1.11412,15.39944,3.55921,1.93937
3,Bound,2.50585,0.17427,0.92003,0.60048,4.97077,2.01022,2.84720,15.12244,4.43190,...,0.00000,11.40550,2.33485,0.24910,0.01274,2.82987,0.97315,6.97970,2.20910,3.05138
4,Bound,0.41722,0.05015,0.16216,0.04995,4.31504,3.95680,1.88518,3.68016,5.69411,...,0.15735,11.21826,2.13359,0.40733,0.00670,1.64880,1.08839,11.05876,2.24214,0.92568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Unbound,1.10593,0.21076,0.81135,2.29406,8.83577,0.25209,0.47947,15.39155,3.91695,...,0.00000,7.29083,2.24943,0.01012,0.03283,1.92170,0.85050,3.60163,5.34013,3.19015
19996,Unbound,1.25224,0.05372,0.57348,2.95141,7.13061,2.14244,1.57490,13.17820,3.37422,...,0.14906,8.23919,2.50342,0.07037,0.08322,3.40546,1.05903,3.53414,5.53222,3.93901
19997,Unbound,0.37885,0.10768,0.37884,0.24213,6.17070,1.33350,1.38840,14.62814,4.50541,...,0.50145,12.35852,3.98299,0.00000,0.00000,1.68247,0.98212,2.66818,1.97313,1.71491
19998,Unbound,0.32093,0.01004,1.57925,2.09424,4.58364,0.72153,0.57340,13.00290,3.62412,...,0.01726,9.24445,2.20758,0.00000,0.38243,3.39166,0.60722,3.90921,6.16538,5.60229


In [10]:
# Instantiate the model.
ml_model = model_building.ClassificationModel(
    dataset=supervised_dataset.df_filtered,
    evaluation_split_ratio=0.15,
    classes_to_use=["Bound", "Unbound"], 
    models_to_use=["CatBoost", "XGBoost", "Random_Forest"],
    scaling_method="min_max",
    out_dir=out_dir, 
    cross_validation_splits=5, 
    cross_validation_repeats=3,
    search_approach="none",
)


Below is a summary of the machine learning you have planned.
You will use 5-fold cross validation and perform 3 repeats.
You will use up to 267 features to build each model, with 85.0% of your data used for training the model, which is 17000 observations. 
15.0% of your data will be used for evaluating the best models produced by the 5-fold cross validation, which is 3000 observations.
You have selected to build 3 machine learning model(s), with the following hyperparameters: 
 
A CatBoost model, with grid search parameters: 
{'iterations': [100]} 

A XGBoost model, with grid search parameters: 
{'n_estimators': [100]} 

A Random_Forest model, with grid search parameters: 
{'n_estimators': [100], 'max_depth': [100]} 

If you're happy with the above, lets get model building!


In [11]:
ml_model.build_models(save_models=True)

Learning rate set to 0.259391
0:	learn: 0.5177789	total: 209ms	remaining: 20.7s
1:	learn: 0.3993231	total: 240ms	remaining: 11.8s
2:	learn: 0.3487314	total: 273ms	remaining: 8.81s
3:	learn: 0.3043576	total: 305ms	remaining: 7.32s
4:	learn: 0.2774604	total: 339ms	remaining: 6.43s
5:	learn: 0.2460856	total: 373ms	remaining: 5.84s
6:	learn: 0.2135924	total: 405ms	remaining: 5.38s
7:	learn: 0.1848698	total: 440ms	remaining: 5.06s
8:	learn: 0.1743368	total: 475ms	remaining: 4.8s
9:	learn: 0.1601173	total: 513ms	remaining: 4.62s
10:	learn: 0.1421832	total: 548ms	remaining: 4.43s
11:	learn: 0.1324111	total: 586ms	remaining: 4.3s
12:	learn: 0.1240420	total: 622ms	remaining: 4.16s
13:	learn: 0.1135446	total: 657ms	remaining: 4.04s
14:	learn: 0.1056527	total: 697ms	remaining: 3.95s
15:	learn: 0.1013955	total: 738ms	remaining: 3.88s
16:	learn: 0.0973486	total: 773ms	remaining: 3.77s
17:	learn: 0.0933506	total: 805ms	remaining: 3.67s
18:	learn: 0.0873453	total: 836ms	remaining: 3.56s
19:	learn: 0.

,model,best_params,best_score,best_standard_deviation
0,CatBoost,{'iterations': 100},0.994157,0.001574
1,XGBoost,{'n_estimators': 100},0.992451,0.001483
2,Random_Forest,"{'max_depth': 100, 'n_estimators': 100}",0.976686,0.002246


In [12]:
reports = ml_model.evaluate_models()

Returning classification reports for each model inside a single dictionary


In [13]:
reports["XGBoost"]

,precision,recall,f1-score,support
Bound,0.990826,0.996704,0.993756,1517
Unbound,0.996608,0.99056,0.993575,1483
accuracy,N/A,N/A,0.993667,3000
macro avg,0.993717,0.993632,0.993665,3000
weighted avg,0.993684,0.993667,0.993666,3000


In [14]:
reports["CatBoost"]

,precision,recall,f1-score,support
Bound,0.994079,0.996045,0.995061,1517
Unbound,0.995946,0.993931,0.994938,1483
accuracy,N/A,N/A,0.995,3000
macro avg,0.995012,0.994988,0.994999,3000
weighted avg,0.995002,0.995,0.995,3000


In [15]:
reports["Random_Forest"]

,precision,recall,f1-score,support
Bound,0.975849,0.985498,0.980649,1517
Unbound,0.985014,0.975051,0.980007,1483
accuracy,N/A,N/A,0.980333,3000
macro avg,0.980431,0.980274,0.980328,3000
weighted avg,0.980379,0.980333,0.980332,3000


In [16]:
post_proc = post_proccessing.SupervisedPostProcessor(
    out_dir=out_dir,
)
post_proc.load_models_from_instance(supervised_model=ml_model)

post_proc.get_feature_importance()
post_proc.get_per_res_importance()

# Pymol Results
pymol_projections.project_multiple_per_res_scores(
    all_per_res_scores=post_proc.all_per_residue_scores,
    out_dir=out_dir
)

pymol_projections.project_multiple_per_feature_scores(
    all_feature_scores=post_proc.all_feature_importances,
    numb_features=200,   #"all",
    out_dir=out_dir
)

C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results/CatBoost_Feature_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results/XGBoost_Feature_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results/Random_Forest_Feature_Importances.csv written to disk.
All feature importances written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results/CatBoost_Per_Residue_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results/XGBoost_Per_Residue_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeature

In [17]:
post_proc.all_feature_importances

{'CatBoost': {'19Arg 83Ala Hbond sc-bb': 8.2771648,
  '31Gly 35Glu Hbond bb-bb': 5.53872205,
  '31Gly 37Ile Other bb-sc': 4.5673266,
  '20Gly 23Gly Hbond bb-bb': 4.19664583,
  '13Arg 53Glu Saltbr sc-sc': 4.04849218,
  '32Glu 35Glu Hbond bb-bb': 4.01514953,
  '13Arg 55Arg Hbond sc-sc': 3.40760041,
  '3Leu 6Glu Hbond bb-bb': 2.71087897,
  '19Arg 22Thr Hbond sc-bb': 2.63881275,
  '31Gly 73Hid Hbond bb-bb': 2.52737575,
  '31Gly 101Phe Other bb-sc': 2.23410528,
  '8Ile 110Ser Hbond bb-bb': 2.08343515,
  '55Arg 93Tyr Hbond sc-sc': 2.0799292,
  '29Ile 73Hid Hbond bb-sc': 2.01068353,
  '104Asn 115Thr Hbond bb-bb': 1.86576068,
  '7Asp 110Ser Hbond sc-bb': 1.80988525,
  '31Gly 72Ser Hbond bb-bb': 1.79342185,
  '31Gly 71Ala Other bb-bb': 1.71462609,
  '19Arg 81Lys Hbond sc-bb': 1.68624612,
  '7Asp 112Arg Saltbr sc-sc': 1.63311217,
  '34Gly 70Asn Hbond bb-sc': 1.63050333,
  '8Ile 112Arg Hbond sc-sc': 1.41222904,
  '77Ala 81Lys Hbond bb-sc': 1.40183701,
  '5Glu 112Arg Saltbr sc-sc': 1.36653694,
  '

In [5]:
# Original version, where use all heavy atoms. 
from key_interactions_finder import utils 
pdb_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\PDZ_pep_Bound_postleap.pdb"
out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results\PDZ_Dist_to_Binding_Site.csv"
distance_array = utils.per_residue_distance_to_site(
    pdb_file=pdb_file,
    out_file=out_file,
    site_defintion="not name H* and resid 118 to 123",
    first_residue=1,
    last_residue=117,
)

# Other version which matches paper I'm comparing to better by using only each residues side chain to measure distances.  
out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results\PDZ_SideChain_Dist_to_Binding_Site.csv"
distance_array = utils.per_residue_distance_to_site(
    pdb_file=pdb_file,
    out_file=out_file,
    site_defintion="not name H* and resid 118 to 123",
    side_chain_only=True,
    first_residue=1,
    last_residue=117,
)


c:\anaconda3\envs\ML_Py3_8\lib\site-packages\MDAnalysis\topology\PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results\PDZ_Dist_to_Binding_Site.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results\PDZ_SideChain_Dist_to_Binding_Site.csv written to disk.


## Re-run, not including any BB-BB interactions. 

This will allow us to focus on more mutatable positions. 

In [2]:
# all input files 
in_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Datasets\PDZ_Combined"
out_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results_no_backbone"

pycontact_files_horizontal = ["PyContact_Per_Frame_Interactions_Block1.csv", "PyContact_Per_Frame_Interactions_Block2.csv",
                              "PyContact_Per_Frame_Interactions_Block3.csv", "PyContact_Per_Frame_Interactions_Block4.csv",
                              "PyContact_Per_Frame_Interactions_Block5.csv", "PyContact_Per_Frame_Interactions_Block6.csv",
                              "PyContact_Per_Frame_Interactions_Block7.csv", "PyContact_Per_Frame_Interactions_Block8.csv",
                              "PyContact_Per_Frame_Interactions_Block9.csv", "PyContact_Per_Frame_Interactions_Block10.csv",
                              "PyContact_Per_Frame_Interactions_Block11.csv", "PyContact_Per_Frame_Interactions_Block12.csv",
                              ]

# dataset prep parts. 
pycontact_dataset = pycontact_processing.PyContactInitializer(
    pycontact_files=pycontact_files_horizontal,
    multiple_files=True,
    merge_files_method="horizontal",  
    remove_false_interactions=True,
    in_dir=in_dir,
)

# Create and save a target file (to load back in next step)
# 10,000 frames were used for both states  
classification_col = ["States"] + ["Bound"] * 10000 + ["Unbound"] * 10000

classifications_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Datasets\PDZ_Classifications_Combined.dat"

with open(classifications_file, "w") as file_out:
    for item in classification_col:
        file_out.write(item + "\n")


# Now generate an instance of the SupervisedFeatureData class and load datasets in.
supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=pycontact_dataset.prepared_df,
    target_file=classifications_file,
    is_classification=True,
    header_present=True 
)

supervised_dataset.df_processed

Your PyContact file(s) have been succefully processed.
You have 914 features and 20000 observations.
The fully processed dataframe is accesible from the '.prepared_df' class attribute.
Your PyContact features and target variable have been succesufully merged.
You can access this dataset through the class attribute: '.df_processed'.


,Target,1Ace 6Glu Hbond bb-sc,2Phe 6Glu Hbond bb-bb,2Phe 5Glu Hbond bb-bb,3Leu 6Glu Hbond bb-bb,3Leu 8Ile Hydrophobic sc-sc,3Leu 7Asp Hbond bb-bb,3Leu 9Pro Hydrophobic sc-sc,3Leu 55Arg Hbond bb-sc,3Leu 112Arg Hbond bb-sc,...,101Phe 106Arg Other bb-sc,108Asn 115Thr Other sc-bb,101Phe 116Asn Other bb-sc,109Ser 114Val Other sc-sc,101Phe 105Ser Other bb-sc,113Ile 116Asn Hbond bb-sc,114Val 117Nme Hbond bb-bb,113Ile 117Nme Hbond bb-bb,112Arg 117Nme Other sc-sc,112Arg 116Asn Other sc-sc
0,Bound,1.43052,2.63266,1.02764,4.04431,1.46040,0.66545,0.17311,2.71112,1.83761,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bound,0.00000,0.32276,0.00000,2.24340,0.43224,1.24919,1.18507,0.14182,1.96343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bound,0.00000,0.52178,0.00000,0.90360,2.21138,1.38761,0.50366,1.53181,2.67639,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bound,0.00000,0.17202,0.00000,4.46798,0.00000,0.00000,0.00000,0.04288,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bound,0.00000,2.96892,0.00000,6.48986,0.36254,0.16546,0.00000,0.95956,1.81796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Unbound,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,Unbound,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,Unbound,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,Unbound,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Filtering 
supervised_dataset.reset_filtering()
supervised_dataset.filter_by_occupancy_by_class(min_occupancy=25)
supervised_dataset.filter_by_main_or_side_chain(
    main_side_chain_types_included=["sc-sc", "bb-sc", "sc-bb"] # "bb-bb" 
)
supervised_dataset.filter_by_avg_strength(average_strength_cut_off=0.25)
print(f"Number of features after filtering: {len(supervised_dataset.df_filtered.columns)}")

Number of features after filtering: 215


In [8]:
# Instantiate the model.
ml_model = model_building.ClassificationModel(
    dataset=supervised_dataset.df_filtered,
    evaluation_split_ratio=0.15,
    classes_to_use=["Bound", "Unbound"], 
    models_to_use=["CatBoost", "XGBoost", "Random_Forest"],
    scaling_method="min_max",
    out_dir=out_dir, 
    cross_validation_splits=5, 
    cross_validation_repeats=3,
    search_approach="none",
)


Below is a summary of the machine learning you have planned.
You will use 5-fold cross validation and perform 3 repeats.
You will use up to 215 features to build each model, with 85.0% of your data used for training the model, which is 17000 observations. 
15.0% of your data will be used for evaluating the best models produced by the 5-fold cross validation, which is 3000 observations.
You have selected to build 3 machine learning model(s), with the following hyperparameters: 
 
A CatBoost model, with grid search parameters: 
{'iterations': [100]} 

A XGBoost model, with grid search parameters: 
{'n_estimators': [100]} 

A Random_Forest model, with grid search parameters: 
{'n_estimators': [100], 'max_depth': [100]} 

If you're happy with the above, lets get model building!


In [9]:
ml_model.build_models(save_models=True)

Learning rate set to 0.259391
0:	learn: 0.5159222	total: 215ms	remaining: 21.3s
1:	learn: 0.4290098	total: 309ms	remaining: 15.2s
2:	learn: 0.3734760	total: 393ms	remaining: 12.7s
3:	learn: 0.3227430	total: 453ms	remaining: 10.9s
4:	learn: 0.2892809	total: 526ms	remaining: 10s
5:	learn: 0.2621408	total: 596ms	remaining: 9.34s
6:	learn: 0.2455069	total: 698ms	remaining: 9.28s
7:	learn: 0.2134690	total: 776ms	remaining: 8.93s
8:	learn: 0.2008538	total: 862ms	remaining: 8.72s
9:	learn: 0.1893002	total: 965ms	remaining: 8.68s
10:	learn: 0.1758463	total: 1.05s	remaining: 8.48s
11:	learn: 0.1688192	total: 1.16s	remaining: 8.51s
12:	learn: 0.1604040	total: 1.26s	remaining: 8.43s
13:	learn: 0.1541294	total: 1.38s	remaining: 8.45s
14:	learn: 0.1406925	total: 1.47s	remaining: 8.3s
15:	learn: 0.1307702	total: 1.53s	remaining: 8.06s
16:	learn: 0.1213335	total: 1.61s	remaining: 7.89s
17:	learn: 0.1156710	total: 1.7s	remaining: 7.77s
18:	learn: 0.1129928	total: 1.78s	remaining: 7.59s
19:	learn: 0.10

,model,best_params,best_score,best_standard_deviation
0,CatBoost,{'iterations': 100},0.992412,0.001258
1,XGBoost,{'n_estimators': 100},0.990745,0.001275
2,Random_Forest,"{'max_depth': 100, 'n_estimators': 100}",0.974608,0.002120


In [10]:
reports = ml_model.evaluate_models()

Returning classification reports for each model inside a single dictionary


In [11]:
reports["XGBoost"]

,precision,recall,f1-score,support
Bound,0.986037,0.992637,0.989326,1494
Unbound,0.992647,0.986056,0.98934,1506
accuracy,N/A,N/A,0.989333,3000
macro avg,0.989342,0.989346,0.989333,3000
weighted avg,0.989355,0.989333,0.989333,3000


In [12]:
reports["Random_Forest"]

,precision,recall,f1-score,support
Bound,0.972812,0.981928,0.977348,1494
Unbound,0.981903,0.972776,0.977318,1506
accuracy,N/A,N/A,0.977333,3000
macro avg,0.977358,0.977352,0.977333,3000
weighted avg,0.977376,0.977333,0.977333,3000


In [13]:
reports["CatBoost"]

,precision,recall,f1-score,support
Bound,0.988689,0.994645,0.991658,1494
Unbound,0.994656,0.988712,0.991675,1506
accuracy,N/A,N/A,0.991667,3000
macro avg,0.991673,0.991679,0.991667,3000
weighted avg,0.991685,0.991667,0.991667,3000


In [14]:
out_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results_no_backbone"

post_proc = post_proccessing.SupervisedPostProcessor(
    out_dir=out_dir,
)
post_proc.load_models_from_instance(supervised_model=ml_model)

post_proc.get_feature_importance()
post_proc.get_per_res_importance()

# Pymol Results
pymol_projections.project_multiple_per_res_scores(
    all_per_res_scores=post_proc.all_per_residue_scores,
    out_dir=out_dir
)

pymol_projections.project_multiple_per_feature_scores(
    all_feature_scores=post_proc.all_feature_importances,
    numb_features=200,   #"all",
    out_dir=out_dir
)

C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results_no_backbone/CatBoost_Feature_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results_no_backbone/XGBoost_Feature_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results_no_backbone/Random_Forest_Feature_Importances.csv written to disk.
All feature importances written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results_no_backbone/CatBoost_Per_Residue_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PDZ_ML\Raw_Results_no_backbone/XGBoost_Per_Residue_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgr